## 성장성을 기준으로 가설검정하여 등급 라벨링

### 1. 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

df = pd.read_csv(r"C:\Users\dlwlg\Desktop\최종프로젝트\all_data_raw\머신러닝\서울시상권데이터.csv")

### 2. 숫자형 데이터만 선택

In [20]:
features = ['당월_매출', '총_유동인구_수', '점포_수', '개업율', '폐업률']
data = df[['상권_코드_명'] + features].dropna()

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data[features])


### 3. 머신러닝 기반 변수 중요도 평가 (예: 랜덤 포레스트 회귀 모델)

In [22]:
# 더미 타겟 변수 생성 (실제로는 성장성 관련 지표를 사용할 수 있음)
y_dummy = np.random.rand(X_scaled.shape[0])

rf = RandomForestRegressor(random_state=42)
rf.fit(X_scaled, y_dummy)
importances = rf.feature_importances_

# 변수 중요도 출력
for feature, importance in zip(features, importances):
    print(f"{feature}: {importance:.3f}")

당월_매출: 0.398
총_유동인구_수: 0.375
점포_수: 0.143
개업율: 0.038
폐업률: 0.046


### 4. 선택된 변수로 성장률(또는 관련 지표) 산출

In [23]:
growth_rate = data[features].mean(axis=1)

### 5. KMeans 클러스터링을 통해 라벨링

In [24]:
X_for_clustering = growth_rate.values.reshape(-1, 1)
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_for_clustering)

# 클러스터 중심의 크기에 따라 라벨 매핑
centers = kmeans.cluster_centers_
cluster_order = np.argsort(centers.flatten())
label_mapping = {cluster_order[0]: '쇠퇴', cluster_order[1]: '정체', cluster_order[2]: '성장'}
labels = [label_mapping[c] for c in clusters]

In [25]:
# 결과 DataFrame 생성
result = pd.DataFrame({
    '상권_코드_명': data['상권_코드_명'].values,
    '가상_성장률': growth_rate.values,
    '라벨': labels
})

print(result.head())

   상권_코드_명        가상_성장률  라벨
0      홍익대  3.437587e+06  쇠퇴
1     가로수길  1.848618e+07  쇠퇴
2      오금역  6.709719e+06  쇠퇴
3  서초3동사거리  1.627271e+07  쇠퇴
4   시청역_8번  5.253277e+06  쇠퇴


# 영업력 활용 변수

#### 가설검정을 활용하여 변수 추려내기

In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 1. CSV 파일 불러오기
df = pd.read_csv(r"C:\Users\dlwlg\Desktop\최종프로젝트\all_data_raw\머신러닝\종합_대시보드데이터.csv")

# 2. 타깃 변수 및 후보 feature 설정
# "당월_매출"을 타깃으로 하고, 나머지 수치형 컬럼(숫자형 데이터)들을 후보로 사용
target = "당월_매출_금액"

# 수치형 컬럼만 선택한 후 타깃 변수 제거
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if target in numeric_cols:
    numeric_cols.remove(target)
candidate_features = numeric_cols
print("후보 feature:")
print(candidate_features)

# 필수 변수에 결측치가 있을 경우 제거
df = df.dropna(subset=candidate_features + [target])

# 3. 독립변수(X)와 종속변수(y) 설정
X = df[candidate_features]
y = df[target]

# 4. 단계별 변수 선택 (Stepwise selection) 함수 정의
def stepwise_selection(X, y, initial_list=[], threshold_in=0.05, threshold_out=0.05, verbose=True):
    """
    단계별 변수 선택 (Forward + Backward elimination)
    - forward step: 아직 포함되지 않은 변수 중 현재 모델에 추가했을 때 p‑value가 threshold_in 보다 작은 변수 추가
    - backward step: 이미 포함된 변수 중 p‑value가 threshold_out 보다 큰 변수를 제거
    최종적으로 p‑value 기준(0.05 이하)을 만족하는 변수 집합을 반환합니다.
    """
    included = list(initial_list)
    while True:
        changed = False
        
        # forward step: 아직 포함되지 않은 변수들 중 추가 검토
        excluded = list(set(X.columns) - set(included))
        new_pvals = pd.Series(index=excluded, dtype=float)
        for new_col in excluded:
            model = sm.OLS(y, sm.add_constant(X[included + [new_col]])).fit()
            new_pvals[new_col] = model.pvalues[new_col]
        if not new_pvals.empty:
            best_pval = new_pvals.min()
            if best_pval < threshold_in:
                best_feature = new_pvals.idxmin()
                included.append(best_feature)
                changed = True
                if verbose:
                    print(f"변수 추가: {best_feature:30} (p-value = {best_pval:.6f})")
        
        # backward step: 현재 포함된 변수들 중 제거 검토 (상수항 제외)
        model = sm.OLS(y, sm.add_constant(X[included])).fit()
        # 상수항('const')는 제외하고 변수들의 p-value 고려
        pvals = model.pvalues.iloc[1:]
        worst_pval = pvals.max()
        if worst_pval > threshold_out:
            worst_feature = pvals.idxmax()
            included.remove(worst_feature)
            changed = True
            if verbose:
                print(f"변수 제거: {worst_feature:30} (p-value = {worst_pval:.6f})")
        if not changed:
            break
    return included

# 5. 단계별 선택 수행 (전체 후보 변수를 대상으로 진행, p-value 기준: 0.05)
selected_features = stepwise_selection(X, y, verbose=True)
print("\n최종 선택된 변수 (단계별 선택 결과):")
print(selected_features)

# 6. 최종 선택된 변수들을 사용한 회귀모델 구성 및 결과 확인
X_selected = sm.add_constant(X[selected_features])
final_model = sm.OLS(y, X_selected).fit()
print("\n최종 회귀분석 결과:")
print(final_model.summary())

# 7. 최종 모델에서 p‑value가 0.05 미만인 변수들(상수항 제외)만 필터링
final_vars = final_model.pvalues[final_model.pvalues < 0.05].index.tolist()
final_vars = [var for var in final_vars if var != "const"]
print("\np-value 기준 (0.05 미만) 최종 선택 변수:")
print(final_vars)


후보 feature:
['기준_년분기_코드', '상권_코드', '총_유동인구_수', '경도1', '위도1', '점포_수']
변수 추가: 점포_수                           (p-value = 0.000000)
변수 추가: 위도1                            (p-value = 0.000000)
변수 추가: 총_유동인구_수                       (p-value = 0.000000)
변수 추가: 상권_코드                          (p-value = 0.000000)
변수 추가: 경도1                            (p-value = 0.000001)
변수 추가: 기준_년분기_코드                      (p-value = 0.000001)

최종 선택된 변수 (단계별 선택 결과):
['점포_수', '위도1', '총_유동인구_수', '상권_코드', '경도1', '기준_년분기_코드']

최종 회귀분석 결과:
                            OLS Regression Results                            
Dep. Variable:               당월_매출_금액   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     2887.
Date:                Tue, 18 Feb 2025   Prob (F-statistic):               0.00
Time:                        10:35:05   Log-Likelihood:            -7.8635e+07
No. Obser

### 추려낸 변수로 영업력 지수 계산하기

In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 1. 데이터 불러오기
# CSV 파일이 로컬에 있다면 파일 경로를 사용하세요.
# 예를 들어, 파일명이 "seoulsisanggweonbunseog_sample.csv"라고 가정합니다.
df = pd.read_csv(r"C:\Users\dlwlg\Desktop\최종프로젝트\all_data_raw\머신러닝\종합_대시보드데이터.csv")

# 2. 필요 변수 확인 및 전처리
# 본 예시에서는 '당월_매출'을 중심으로,
# '평균_당월_매출_건수', '평균_주중_매출_금액', '평균_주말_매출_금액'을 독립 변수로 사용합니다.
features = ['기준_년분기_코드','점포_수',  '총_유동인구_수']
target = '당월_매출_금액'

# 결측치 제거(필요한 경우)
df = df.dropna(subset=features + [target])

# 3. 전체 데이터를 대상으로 회귀모델 생성
X = df[features]
y = df[target]
X = sm.add_constant(X)  # 상수항 추가
model = sm.OLS(y, X).fit()

# 회귀모델 결과 요약 출력(가설검정 결과 포함)
print(model.summary())

# 4. 각 관측치에 대해 영업력지수(예측값) 계산
df["영업력지수"] = model.predict(X)

# 만약 상권별(예: "상권_코드_명")로 집계하여 영업력지수를 적용하고 싶다면:
grouped_index = df.groupby("상권_코드_명")["영업력지수"].mean().reset_index()
grouped_index.columns = ["상권_코드_명", "평균_영업력지수"]

print("\n각 상권별 평균 영업력지수:")
print(grouped_index.head())


                            OLS Regression Results                            
Dep. Variable:               당월_매출_금액   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     4883.
Date:                Tue, 18 Feb 2025   Prob (F-statistic):               0.00
Time:                        10:38:42   Log-Likelihood:            -7.8636e+07
No. Observations:             3185332   AIC:                         1.573e+08
Df Residuals:                 3185328   BIC:                         1.573e+08
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -9.17e+10   1.96e+10     -4.675      0.0

## 1. 포화도 라벨링 기준 정하기
**경쟁강도 :: 상권 점포수 / 행정동 점포수 -> 이것만 사용하기로 결정**  
매출경쟁도 :: 상권 당월 매출 / 해당 행정동 당월 매출  
상권밀집도 :: 상권 점포수 / 상권 면적

포화지수 = (경쟁강도 * 0.4) + (매출경쟁도 * 0.4) + (상권밀집도 * 0.2)

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

# 1. 데이터 불러오기
# CSV 파일이 로컬에 있다면 파일 경로를 사용하세요.
# 예를 들어, 파일명이 "seoulsisanggweonbunseog_sample.csv"라고 가정합니다.
df = pd.read_csv(r"C:\Users\m\project\공공데이터_상권분석\data\머신러닝\인공지능용_before.csv")

# 경쟁강도 지수 계산
df['competition_index'] = df['점포_수'] / df.groupby('자치구_코드_명')['점포_수'].transform('sum')

# 정규성 검정
statistic, p_value = stats.shapiro(df['competition_index'])
print("Shapiro-Wilk 검정 결과:")
print(f"통계량: {statistic:.4f}")
print(f"p-value: {p_value:.4f}")

# 단일표본 t-검정 (예: 기준값 0.5와 비교)
t_stat, t_p_value = stats.ttest_1samp(df['competition_index'], 0.5)
print("\n단일표본 t-검정 결과:")
print(f"t-통계량: {t_stat:.4f}")
print(f"p-value: {t_p_value:.4f}")

# 기술통계량 출력
print("\n기술통계량:")
print(df['competition_index'].describe())

Shapiro-Wilk 검정 결과:
통계량: 0.2557
p-value: 0.0000

단일표본 t-검정 결과:
t-통계량: -411029.5863
p-value: 0.0000

기술통계량:
count    139700.000000
mean          0.000179
std           0.000455
min           0.000000
25%           0.000044
50%           0.000086
75%           0.000175
max           0.025066
Name: competition_index, dtype: float64


c:\Users\m\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 139700.
  res = hypotest_fun_out(*samples, **kwds)
